In [ ]:
import numpy as np
import soundfile as sf
from scipy.interpolate import interp1d
import pandas as pd
from scipy.fft import rfft, irfft, rfftfreq
import matplotlib.pyplot as plt
import os
import gc

# Paths
base_path = "C:/Users/Valer/OneDrive/ODL Hydrophone/ODL Recordings/"
output_fig_spectrogram_folder = "C:/Users/Valer/OneDrive/ODL Hydrophone/Data Sheets/Spectrograms/"
calib_csv_path = "C:/Users/Valer/OneDrive/ODL Hydrophone/Data Sheets/Bins Data for Hydrophone Sensitivity.csv"

os.makedirs(output_fig_spectrogram_folder, exist_ok=True) #creating spectro folder 

calib_df = pd.read_csv(calib_csv_path)
freq_calib = calib_df["Frequency (Hz)"].to_numpy()
sens_calib = calib_df["Sensitivity (dB re 1V/1uPa)"].to_numpy()
sens_calib_gain = sens_calib + 25.0

for filename in os.listdir(base_path):
    if filename.lower().endswith(".wav"):
        print(f"Processing spectrogram for: {filename}")
        wav_path = os.path.join(base_path, filename)
        signal_raw, fs = sf.read(wav_path)
        N = len(signal_raw)
        freqs = rfftfreq(N, d=1/fs)

        interp_func = interp1d(freq_calib, sens_calib_gain, kind='linear', fill_value='extrapolate')
        sens_interp_dB = interp_func(freqs)
        gain_correction = 10 ** (-sens_interp_dB / 20)

        signal_fft = rfft(signal_raw)
        calibrated_fft = signal_fft * gain_correction
        signal_calibrated = irfft(calibrated_fft)

        NFFT = 65536
        fig, ax = plt.subplots(figsize=(14, 6))
        pxx, freqs_spec, bins_spec, im = ax.specgram(
            signal_calibrated,
            Fs=fs,
            NFFT=NFFT,
            noverlap=NFFT // 2,
            vmin=20,
            vmax=120
        )
        ax.set_title(f"Spectrogram of {filename}")
        ax.set_ylabel("Frequency (Hz)")
        ax.set_xlabel("Time (s)")
        ax.set_ylim(100, 700)
        cbar = fig.colorbar(im, ax=ax, label="Spectral Level (dB re 1 µPa²/Hz)")
        cbar.ax.yaxis.set_label_position('right')
        plt.tight_layout()

        fig_name = f"spectrogram_{os.path.splitext(filename)[0]}.png"
        fig_path = os.path.join(output_fig_spectrogram_folder, fig_name)
        plt.savefig(fig_path)
        plt.close()
        gc.collect()
        print(f"Spectrogram saved to: {fig_path}")

        ##ADD FREE MEMORY TO SPECTRO TOO!:
        del signal_raw, signal_fft, calibrated_fft, signal_calibrated
        del pxx, freqs_spec, bins_spec, im
        del fig, ax, cbar